# Optimize LSTM model with MACD and forcasting highs for SPY

In [1]:
!pip install datasets
!pip install transformers
!pip install huggingface_hub
!pip install git+https://github.com/huggingface/peft.git
!pip install git+https://github.com/huggingface/accelerate.git
!pip install torch
!pip install googlesearch-python
!pip install GoogleNews
!pip install textblob
!pip install schedule
!pip install time
!pip install telebot
!pip install telethon
!pip install transformers
!pip install vadersentiment
!pip install alpha_vantage
!pip install Prophet
!pip install tensorflow

  Cloning https://github.com/huggingface/peft.git to /private/var/folders/jp/6l9fcsfs0dq5_695_52tn60r0000gn/T/pip-req-build-xg47rwso
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /private/var/folders/jp/6l9fcsfs0dq5_695_52tn60r0000gn/T/pip-req-build-xg47rwso
  Resolved https://github.com/huggingface/peft.git to commit e72a96f7cfdb330236c1b6dfe49be398d83f089e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/huggingface/accelerate.git to /private/var/folders/jp/6l9fcsfs0dq5_695_52tn60r0000gn/T/pip-req-build-e41n3ti1
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate.git /private/var/folders/jp/6l9fcsfs0dq5_695_52tn60r0000gn/T/pip-req-build-e41n3ti1
  Resolved https://github.com/huggingface/accelerate.git to commit 12a007d55937345aa986f5d7b1a1b6f2038465a7
  Installing build dependen

In [2]:
!pip install optuna

In [3]:
import requests
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
import schedule
import time
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from GoogleNews import GoogleNews
import yfinance as yf
import torch
import torch.nn.functional as F
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from prophet import Prophet
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Dropout
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

2024-07-12 05:14:09.415200: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def get_numeric_data(ticker):
    # Define the ticker symbol and time period
    period = "6mo" #must be one of ['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']"
    interval = "1h"

    # Retrieve the historical market data
    data = yf.download(tickers=ticker, period=period, interval=interval)

    # Retrieve the ticker info
    ticker_info = yf.Ticker(ticker).info

    # Extract the market cap
    #market_cap = ticker_info['marketCap']

    # Calculate the moving averages
    data['MA_50'] = data['Close'].rolling(window=50).mean()
    data['MA_200'] = data['Close'].rolling(window=200).mean()

    # Calculate the RSI
    delta = data['Close'].diff(1)
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up = up.rolling(window=14).mean()
    roll_down = down.rolling(window=14).mean().abs()
    RS = roll_up / roll_down
    data['RSI'] = 100.0 - (100.0 / (1.0 + RS))

    # Calculate the MACD
    data['EMA_12'] = data['Close'].ewm(span=12, adjust=False).mean()
    data['EMA_26'] = data['Close'].ewm(span=26, adjust=False).mean()
    data['MACD'] = data['EMA_12'] - data['EMA_26']
    data['Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()

    # Create a single dataframe called numeric_df
    numeric_df = data.select_dtypes(include=['number'])

    # Reset the index
    numeric_df.reset_index(inplace=True)

    return numeric_df

ticker = 'SPY'
numeric = get_numeric_data(ticker)
numeric.columns

[*********************100%%**********************]  1 of 1 completed


Index(['Datetime', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
       'MA_50', 'MA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'Signal'],
      dtype='object')

# Initial testing

## Train

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

def train_lstm_forcast(df):
    # Load the data
    df = df.copy()
    #df.drop('headline', axis = 1, inplace = True)

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the relevant columns
    relevant_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
                       'MA_50', 'MA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'Signal']
    data_df = df[relevant_cols]

    # Drop any rows with missing values
    #df_resampled.dropna(inplace=True)

    # Create a new dataframe with only the Close column
    close_df = data_df[['High']]
    close_df.set_index(df['Datetime'], inplace=True)

    # Scale the data using Min-Max Scaler
    scaler = MinMaxScaler(feature_range=(0,1))
    close_scaled = scaler.fit_transform(close_df)

    # Create a function to create the LSTM dataset
    def create_lstm_dataset(X, y, time_steps=1):
        Xs, ys = [], []
        for i in range(len(X)-time_steps):
          Xs.append(X[i:(i+time_steps)])
          ys.append(y[i+time_steps])
        return np.array(Xs), np.array(ys)

    # Create the LSTM dataset
    X, y = create_lstm_dataset(close_scaled, close_scaled, time_steps=60)

    # Reshape the data for LSTM
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # Create the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)

    model.fit(X, y, epochs=100, batch_size=32, verbose=1,
              validation_split=0.2, callbacks=[early_stopping])
    
    model.save('lstm_model.keras')

    # Make predictions on the last 60 hours of data
    last_60_hours = close_scaled[-60:]
    last_60_hours = last_60_hours.reshape(1, 60, 1)
    prediction = model.predict(last_60_hours)

    # Inverse transform the prediction
    prediction = scaler.inverse_transform(prediction)

    # Print the predicted close price
    print("Predicted High Price:", prediction[0][0])

    # Plot the predicted close price
    plt.plot(close_df.index[-60:], close_df.values[-60:])
    plt.plot([close_df.index[-1] + pd.Timedelta(hours=1)], [prediction[0][0]], 'ro', label='Predicted Close Price')
    plt.legend(loc='upper left')
    plt.show()

    return prediction

ticker = 'SPY'
df = get_numeric_data(ticker)
train_lstm_forcast(df)

[*********************100%%**********************]  1 of 1 completed
/Users/aidanlew/miniconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100


In [23]:
def lstm_forcast(df, future_days):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the relevant columns
    relevant_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
                       'MA_50', 'MA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'Signal']
    data_df = df[relevant_cols]

    # Drop any rows with missing values
    #df_resampled.dropna(inplace=True)

    # Create a new dataframe with only the Close column
    close_df = data_df[['High']]
    close_df.set_index(df['Datetime'], inplace=True)

    # Scale the data using Min-Max Scaler
    scaler = MinMaxScaler(feature_range=(0,1))
    close_scaled = scaler.fit_transform(close_df)

    # Create a function to create the LSTM dataset
    def create_lstm_dataset(X, y, time_steps=1):
        Xs, ys = [], []
        for i in range(len(X)-time_steps):
          Xs.append(X[i:(i+time_steps)])
          ys.append(y[i+time_steps])
        return np.array(Xs), np.array(ys)

    # Create the LSTM dataset
    X, y = create_lstm_dataset(close_scaled, close_scaled, time_steps=60)

    # Reshape the data for LSTM
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # Create the LSTM model
    model = Sequential()
    model.add(LSTM(units=50, return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer='adam', loss='mean_squared_error')

    # Load the trained model
    import tensorflow as tf
    model = tf.keras.models.load_model('lstm_model.keras')

    # Prepare the input data for forecasting
    last_60_hours = close_scaled[-60:]
    future_input = last_60_hours.reshape(1, 60, 1)

    # Forecast future close prices
    future_close_prices = []
    for i in range(future_days):
        prediction = model.predict(future_input)
        prediction = scaler.inverse_transform(prediction)
        future_close_prices.append(prediction[0, 0])
        future_input = np.append(future_input[:, 1:, :], prediction.reshape((1, 1, 1)), axis=1)

    return future_close_prices

ticker = 'SPY'
df = get_numeric_data(ticker)
lstm_forcast(df, future_days = 30)

[*********************100%%**********************]  1 of 1 completed
/Users/aidanlew/miniconda3/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 229ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━

[555.7855,
 569.37964,
 579.1692,
 586.8132,
 592.7993,
 597.51495,
 601.2437,
 604.20056,
 606.5503,
 608.42084,
 609.91223,
 611.10315,
 612.05585,
 612.8196,
 613.4334,
 613.92834,
 614.3287,
 614.65405,
 614.91956,
 615.1373,
 615.3171,
 615.4662,
 615.59064,
 615.6952,
 615.78345,
 615.8584,
 615.9223,
 615.97705,
 616.0242,
 616.06464]

# Optimization

In [ ]:
import optuna

ticker = 'SPY'
df = get_numeric_data(ticker)

def lstm_forcast_trial(df, future_days, trial):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the relevant columns
    relevant_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
                       'MA_50', 'MA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'Signal']
    data_df = df[relevant_cols]

    # Drop any rows with missing values
    #df_resampled.dropna(inplace=True)

    # Create a new dataframe with only the Close column
    close_df = data_df[['High']]
    close_df.set_index(df['Datetime'], inplace=True)

    # Scale the data using Min-Max Scaler
    scaler = MinMaxScaler(feature_range=(0,1))
    close_scaled = scaler.fit_transform(close_df)

    #  Create a function to create the LSTM dataset

    def create_lstm_dataset(X, y, time_steps=1):
        Xs, ys = [], []
        for i in range(len(X)-time_steps):
          Xs.append(X[i:(i+time_steps)])
          ys.append(y[i+time_steps])
        return np.array(Xs), np.array(ys)

    # Create the LSTM dataset
    X, y = create_lstm_dataset(close_scaled, close_scaled, time_steps=trial.suggest_int('time_steps', 30, 90))
    # Split the data into training and testing sets
    train_size = int(0.8 * len(close_scaled))
    X_train, X_test = close_scaled[0:train_size], close_scaled[train_size:]
    y_train, y_test = close_scaled[0:train_size], close_scaled[train_size:]

    # Create the LSTM dataset
    X_train, y_train = create_lstm_dataset(X_train, y_train, time_steps=trial.suggest_int('time_steps', 30, 90))
    X_test, y_test = create_lstm_dataset(X_test, y_test, time_steps=trial.suggest_int('time_steps', 30, 90))

    # Reshape the data for LSTM
    X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

    # Create the LSTM model
    model = Sequential()
    model.add(LSTM(units=trial.suggest_int('units', 20, 100), return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(Dropout(trial.suggest_uniform('dropout', 0.1, 0.5)))
    model.add(LSTM(units=trial.suggest_int('units', 20, 100), return_sequences=False))
    model.add(Dropout(trial.suggest_uniform('dropout', 0.1, 0.5)))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer=trial.suggest_categorical('optimizer', ['adam', 'rmsprop']), loss='mean_squared_error')

    # Train the model
    model.fit(X_train, y_train, epochs=trial.suggest_int('epochs', 10, 50), batch_size=trial.suggest_int('batch_size', 16, 128), verbose=1)

    # Forecast future close prices
    future_input = X_test[-1].reshape(1, X_test.shape[1], 1)
    future_close_prices = []
    for i in range(future_days):
        prediction = model.predict(future_input)
        prediction = scaler.inverse_transform(prediction)
        future_close_prices.append(prediction[0, 0])
        future_input = np.append(future_input[:, 1:, :], prediction.reshape((1, 1, 1)), axis=1)

    return future_close_prices, y_test

def objective(trial):
    future_close_prices, y_test = lstm_forcast_trial(df, future_days=30, trial=trial)
    # Calculate the mean absolute error (MAE) as the objective function
    mae = np.mean(np.abs(np.array(future_close_prices) - y_test[-30:]))
    return mae

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=1000)

print('Best trial:', study.best_trial)
print('Best parameters:', study.best_params)
print('Best value:', study.best_value)

def send_notification():
    BOT_TOKEN = "7026191163:AAE4CiZV_DLachLeHqz4xAVzDDymqOIKz8Y" ######## Token ##########
    chat_id = "7298484676" ######## Token ##########

    message = f"Done trainig {study.best_params} {study.best_value}" 

    url = f"https://api.telegram.org/bot{BOT_TOKEN}/sendMessage?chat_id={chat_id}&text={message}"

    requests.get(url).json()

send_notification()

# Testing

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
from tensorflow.keras.callbacks import EarlyStopping

def train_lstm_forcast(df):
    # Load the data
    df = df.copy()
    #df.drop('headline', axis = 1, inplace = True)

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])
    

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the relevant columns
    relevant_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
                       'MA_50', 'MA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'Signal']
    data_df = df[relevant_cols]

    # Drop any rows with missing values
    #df_resampled.dropna(inplace=True)

    # Create a new dataframe with only the Close column
    close_df = data_df[['High']]
    close_df.set_index(df['Datetime'], inplace=True)

    # Scale the data using Min-Max Scaler
    scaler = MinMaxScaler(feature_range=(0,1))
    close_scaled = scaler.fit_transform(close_df)

    # Create a function to create the LSTM dataset
    def create_lstm_dataset(X, y, time_steps=1):
        Xs, ys = [], []
        for i in range(len(X)-time_steps):
          Xs.append(X[i:(i+time_steps)])
          ys.append(y[i+time_steps])
        return np.array(Xs), np.array(ys)

    # Create the LSTM dataset
    X, y = create_lstm_dataset(close_scaled, close_scaled, time_steps=60)

    # Reshape the data for LSTM
    X = X.reshape(X.shape[0], X.shape[1], 1)

    # Create the LSTM model
    params = {'time_steps': 87,
              'units': 27,
              'dropout': 0.477996409662855,
              'optimizer': 'rmsprop',
              'epochs': 10,
              'batch_size': 116
              }
              
    model = Sequential()
    model.add(LSTM(units=params['units'], return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(Dropout(params['dropout']))
    model.add(LSTM(units=params['units'], return_sequences=False))
    model.add(Dropout(params['dropout']))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer=params['optimizer'], loss='mean_squared_error')

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.001)

    model.fit(X, y, epochs=params['epochs'], batch_size=params['batch_size'], verbose=1,
              validation_split=0.2, callbacks=[early_stopping])
    
    model.save('lstm_model.keras')

    # Make predictions on the last 60 hours of data
    last_60_hours = close_scaled[-60:]
    last_60_hours = last_60_hours.reshape(1, 60, 1)
    prediction = model.predict(last_60_hours)

    # Inverse transform the prediction
    prediction = scaler.inverse_transform(prediction)

    # Print the predicted close price
    print("Predicted High Price:", prediction[0][0])

    # Plot the predicted close price
    plt.plot(close_df.index[-60:], close_df.values[-60:])
    plt.plot([close_df.index[-1] + pd.Timedelta(hours=1)], [prediction[0][0]], 'ro', label='Predicted Close Price')
    plt.legend(loc='upper left')
    plt.show()

    return prediction

ticker = 'SPY'
df = get_numeric_data(ticker)
train_lstm_forcast(df)

In [ ]:
def lstm_forcast(df, future_days):
    # Load the data
    df = df.copy()

    # Convert the Datetime column to datetime format
    df['Datetime'] = pd.to_datetime(df['Datetime'])

    # Move the Datetime column to the first position
    cols = df.columns.tolist()
    cols.insert(0, cols.pop(cols.index('Datetime')))
    df = df[cols]

    # Create a new dataframe with only the relevant columns
    relevant_cols = ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume',
                       'MA_50', 'MA_200', 'RSI', 'EMA_12', 'EMA_26', 'MACD', 'Signal']
    data_df = df[relevant_cols]

    # Drop any rows with missing values
    #df_resampled.dropna(inplace=True)

    # Create a new dataframe with only the Close column
    close_df = data_df[['High']]
    close_df.set_index(df['Datetime'], inplace=True)

    # Scale the data using Min-Max Scaler
    scaler = MinMaxScaler(feature_range=(0,1))
    close_scaled = scaler.fit_transform(close_df)

    # Create a function to create the LSTM dataset
    def create_lstm_dataset(X, y, time_steps=1):
        Xs, ys = [], []
        for i in range(len(X)-time_steps):
          Xs.append(X[i:(i+time_steps)])
          ys.append(y[i+time_steps])
        return np.array(Xs), np.array(ys)

    # Create the LSTM dataset
    X, y = create_lstm_dataset(close_scaled, close_scaled, time_steps=60)

    # Reshape the data for LSTM
    X = X.reshape(X.shape[0], X.shape[1], 1)

  # Create the LSTM model
    params = {'time_steps': 87,
          'units': 27,
          'dropout': 0.477996409662855,
          'optimizer': 'rmsprop',
          'epochs': 10,
          'batch_size': 116
          }
    
    model = Sequential()
    model.add(LSTM(units=params['units'], return_sequences=True, input_shape=(X.shape[1], 1)))
    model.add(Dropout(params['dropout']))
    model.add(LSTM(units=params['units'], return_sequences=False))
    model.add(Dropout(params['dropout']))
    model.add(Dense(units=1))

    # Compile the model
    model.compile(optimizer=params['optimizer'], loss='mean_squared_error')

    # Load the trained model
    import tensorflow as tf
    model = tf.keras.models.load_model('lstm_model.keras')

    # Prepare the input data for forecasting
    last_60_hours = close_scaled[-60:]
    future_input = last_60_hours.reshape(1, 60, 1)

    # Forecast future close prices
    future_close_prices = []
    for i in range(future_days):
        prediction = model.predict(future_input)
        prediction = scaler.inverse_transform(prediction)
        future_close_prices.append(prediction[0, 0])
        future_input = np.append(future_input[:, 1:, :], prediction.reshape((1, 1, 1)), axis=1)
    
    # plot future close prices
    plt.plot( future_close_prices, label='Predicted')
    plt.legend()
    plt.show()
    
    return future_close_prices

ticker = 'SPY'
df = get_numeric_data(ticker)
lstm_forcast(df, future_days = 30)